# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [84]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [62]:
city_file = pd.read_csv("../WeatherPy/outputs/Cities Data.csv")
city_file.head()

,City,Country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,nome,us,64.50,-165.41,48.20,81,90,13.87,1600036035
1,busselton,au,-33.65,115.33,55.99,59,95,5.93,1600035932
2,beidao,cn,34.57,105.89,61.38,84,98,1.72,1600036085
3,saint george,bm,37.10,-113.58,97.00,7,1,11.41,1600036086
4,jamestown,sh,42.10,-79.24,71.01,77,1,11.41,1600036087


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [63]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Customize the size of the figure
figure_layout = {
    'width': '900px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout, center=(0,0), zoom_level=2)

In [64]:
locations = city_file[["Lat","Lon"]].astype(float)
humidities = city_file["Humidity"].astype(float).to_list()

heat_layer = gmaps.heatmap_layer(locations, weights=humidities, dissipating=False, max_intensity=100, point_radius = 3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [65]:
ideal_weather = city_file.loc[
    ((city_file["Max Temp"]) < 80) &
    ((city_file["Max Temp"]) > 70) &
    ((city_file["Wind Speed"]) < 10) &
    ((city_file["Cloudiness"]) == 0)]

len(ideal_weather)

10

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [134]:
hotel_df = ideal_weather.loc[:,["City","Country","Lat","Lon"]]
hotel_df = hotel_df.rename(columns={"Lon":"Lng"})

hotel_df

,City,Country,Lat,Lng
140,darnah,ly,32.77,22.64
247,buriti alegre,br,-18.14,-49.04
264,vazante,br,-17.99,-46.91
334,awbari,ly,26.59,12.78
355,richards bay,za,-28.78,32.04
399,imbituba,br,-28.24,-48.67
427,moissac,fr,44.10,1.09
489,angouleme,fr,45.65,0.15
610,marsa matruh,eg,31.35,27.25
617,aracuai,br,-16.85,-42.07


In [136]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_search = "hotel"
target_radius = 5000
target_type = "hotel"

hotel_names = []

for city in hotel_df["City"]:
    
    city_lat = hotel_df["Lat"].loc[hotel_df["City"]==city].iloc[0] 
    city_lon = hotel_df["Lng"].loc[hotel_df["City"]==city].iloc[0] 
    coords = f"{city_lat},{city_lon}"
    
    # set up a parameters dictionary
    params = {"location": coords,
              "keyword": target_search,
              "radius": target_radius,
              "type": target_type,
              "key": g_key}
    
    # run a request using params dictionary
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_names.append(response["results"][0]["name"])
    except:
        hotel_names.append("NO HOTELS FOUND")
    
hotel_df["Hotel Name"] = hotel_names
hotel_df

,City,Country,Lat,Lng,Hotel Name
140,darnah,ly,32.77,22.64,فندق النعيم
247,buriti alegre,br,-18.14,-49.04,FLIP PALACE HOTEL
264,vazante,br,-17.99,-46.91,Hotel Pousada e Restaurante
334,awbari,ly,26.59,12.78,NO HOTELS FOUND
355,richards bay,za,-28.78,32.04,BON Hotel Waterfront Richards Bay
399,imbituba,br,-28.24,-48.67,Praia Hotel Imbituba
427,moissac,fr,44.10,1.09,Hôtel & Spa Le Moulin De Moissac
489,angouleme,fr,45.65,0.15,Villa Côté Plateau
610,marsa matruh,eg,31.35,27.25,Air Force hotel
617,aracuai,br,-16.85,-42.07,Village das Minas Hotel


In [137]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [138]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

In [141]:
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…